# TERMINAL PRINCIPAL

## IMPORTES

In [22]:
import re
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
import os
import random

## DATOS MONGODB

In [19]:
PATH= "../res/config_mongo.txt"

In [20]:
# Codigo para sacar variables aqui
def get_mongo_data(file_path):

    config = {
        'server': None,
        'port': 27017,  
        'username': None,
        'password': None,
        'database': None,
    }
    
    try:
        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()
                if not line or line.startswith('#'):
                    continue
                
                if '=' in line:
                    key, value = line.split('=', 1)
                    key = key.strip().lower()
                    if key in config:
                        config[key] = value.strip()
    
    except FileNotFoundError:
        raise Exception(f"El archivo de configuración {file_path} no existe")
    except Exception as e:
        raise Exception(f"Error al leer el archivo de configuración: {str(e)}")
    
    if not config['server']:
        raise Exception("El servidor no está configurado en el archivo")
    
    return config

config = get_mongo_data(PATH)
print("Credenciales: ",config)

Credenciales:  {'server': '127.25.0.5', 'port': '27017', 'username': 'user', 'password': 'pass', 'database': 'PIA'}


## FUNCIONES PRINCIPALES

### 1. Comprobar conectividad

In [13]:
def test_mongo_connection(file_path):
    """
    Prueba la conexión a MongoDB usando datos de un archivo TXT
    
    Args:
        file_path (str): Ruta al archivo de configuración
    """
    try:
        config = get_mongo_data(file_path)
        
        uri = f"mongodb://172.25.0.5:27017/PIA"
        
        client = MongoClient(uri, serverSelectionTimeoutMS=5000)
        client.admin.command('ping')
        print("✅ Conexión exitosa a MongoDB")
        print(f"Servidor: {config['server']}")
        print(f"Base de datos: {config['database']}")
        
    except ConnectionFailure as e:
        print(f"❌ Error de conexión a MongoDB: {str(e)}")
    except Exception as e:
        print(f"❌ Error: {str(e)}")

test_mongo_connection(PATH)

✅ Conexión exitosa a MongoDB
Servidor: 127.25.0.5
Base de datos: PIA


### 2. Un coche entra

In [24]:
def seleccionar_foto_aleatoria(carpeta):
    # Listar todos los archivos .jpg en la carpeta
    archivos = [f for f in os.listdir(carpeta) if f.lower().endswith('.jpg')]
    
    if not archivos:
        raise FileNotFoundError("No se encontraron archivos .jpg en la carpeta especificada.")
    
    foto_aleatoria = random.choice(archivos)
    
    return os.path.join(carpeta, foto_aleatoria)

In [62]:
import os
import random
from datetime import datetime
from pymongo import MongoClient
from bson.binary import Binary

PATH_COCHES = "../res/coches"

def seleccionar_foto_aleatoria(carpeta):
    archivos = [f for f in os.listdir(carpeta) if f.lower().endswith('.jpg')]
    if not archivos:
        raise FileNotFoundError("No hay archivos .jpg en la carpeta.")
    foto_aleatoria = random.choice(archivos)
    return os.path.join(carpeta, foto_aleatoria)

def process_car_entry():
    try:
        # PT1. Seleccionar foto aleatoria
        photo_coche = seleccionar_foto_aleatoria(PATH_COCHES)
        print(f"Foto seleccionada: {photo_coche}")

        # PT2. Simular detección de matrícula con YOLO (guardamos en un archivo)
        license_plate_image = "detected_plate.jpg"
        print(f"Matrícula detectada: {license_plate_image}")

        # PT3. Simular OCR (texto fijo)
        license_plate_text = "ABC1234"
        print(f"Texto de la matrícula: {license_plate_text}")

        # Leer imágenes como binarios
        with open(photo_coche, 'rb') as f_photo:
            photo_binary = Binary(f_photo.read())


        # Conexión a MongoDB
        uri = "mongodb://172.25.0.5:27017/"
        client = MongoClient(uri)
        db = client['PIA']
        collection = db["coches"]

        # Crear documento con los binarios
        car_entry = {
            "photo_full": photo_binary,       # Imagen completa en BinData
            "photo_plate": license_plate_image,      # Matrícula en BinData
            "plate_text": license_plate_text,
            "status": "Dentro",
            "entry_timestamp": datetime.now()
        }

        # Insertar en MongoDB
        collection.insert_one(car_entry)
        print("✅ Coche registrado en MongoDB (BinData):", car_entry)

    except FileNotFoundError as e:
        print(f"❌ Error: {e}")
    except Exception as e:
        print(f"❌ Error inesperado: {e}")

process_car_entry()

Foto seleccionada: ../res/coches/eu3.jpg
Matrícula detectada: detected_plate.jpg
Texto de la matrícula: ABC1234
✅ Coche registrado en MongoDB (BinData): {'photo_full': Binary(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00\x84\x00\x10\x0b\x0c\x0e\x0c\n\x10\x0e\r\x0e\x12\x11\x10\x13\x18(\x1a\x18\x16\x16\x181#%\x1d(:3=<9387@H\\N@DWE78PmQW_bghg>Mqypdx\\egc\x01\x11\x12\x12\x18\x15\x18/\x1a\x1a/cB8Bcccccccccccccccccccccccccccccccccccccccccccccccccc\xff\xc0\x00\x11\x08\x01h\x01\xe0\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1b\x00\x00\x01\x05\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\xff\xc4\x00J\x10\x00\x01\x03\x02\x04\x02\x05\x07\t\x07\x02\x04\x06\x03\x00\x00\x01\x00\x02\x03\x04\x11\x05\x12!1AQ\x06\x13"aq\x142\x81\x91\xa1\xb1\xd1\x15#BRSr\x92\xc1\xd2\x163CTb\x82\xe1\x17\xf0$D\x93\xa2\x074c\x83\xb2\xf1%5s\xff\xc4\x00\x19\x01\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\

### 3. Un coche con matricula [X] sale

In [17]:
def process_car_exit(plate_text):
    """
    Procesa la salida de un coche dado su matrícula.
    
    Args:
        plate_text (str): Texto de la matrícula del coche que sale.
    """
    try:

        uri = f"mongodb://172.25.0.5:27017/PIA"
        client = MongoClient(uri)
        db = client[config['database']]
        collection = db["cars"]

        car = collection.find_one({"plate_text": plate_text, "status": "Dentro"})
        
        if car:
            collection.update_one(
                {"_id": car["_id"]},
                {"$set": {"status": "Fuera", "exit_timestamp": datetime.now()}}
            )
            print(f"✅ Coche con matrícula {plate_text} marcado como 'Fuera'")
        else:
            print(f"❌ No se encontró un coche con matrícula {plate_text} y estado 'Dentro'")
    
    except Exception as e:
        print(f"❌ Error al procesar la salida del coche: {str(e)}")

# Ejemplo de uso


In [42]:
# CODIGO AQUI
matricula = input("Introduce la matricula del coche que sale: ")
process_car_exit(matricula)




❌ No se encontró un coche con matrícula 4345345 y estado 'Dentro'
